In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import *
import math

### Read Component Properties:  

In [2]:
component_properties = pd.read_excel("Componentproperties.xlsx","componentproperties")
component_properties

,component,molecular_weight,Tbi,Pci,Tci
0,Methane,16.0430,-258.730,666.4,-116.67
1,Ethane,30.0700,-127.490,706.5,89.92
2,Propane,44.1000,-43.750,616.0,206.06
3,iso-Butane,58.1200,10.780,527.9,274.46
4,n-Butane,58.1200,31.080,550.6,305.62
5,iso-Pentane,72.1500,82.120,490.4,369.10
6,n-Pentane,72.1500,96.920,488.6,385.80
7,neo-Pentane,72.1500,49.100,464.0,321.13
8,n-Hexane,86.1800,155.720,436.9,453.60
9,Carbondioxide,44.0100,-109.257,1071.0,87.91


### Read Plus Fraction Properties: 

In [3]:
plus_parameter = pd.read_excel("Componentproperties.xlsx",'plusparameter')
plus_parameter

,Parameter,a,b,c,d,e,f
0,Tc,544.40000,0.299800,1.05550,-0.000135,-0.616410,0.000000
1,Pc,45203.00000,-0.806300,1.60150,-0.001808,-0.308400,0.000000
2,Vc,0.01206,0.203780,-1.30360,-0.002657,0.528700,0.002601
3,Tb,6.77857,0.401673,-1.58262,0.003774,2.984036,-0.004253


### Read Composition: 

In [4]:
composition = pd.read_excel("Componentproperties.xlsx", 'composition')
composition

,Component,Zi
0,Methane,0.15
1,Ethane,0.17
2,Propane,0.15
3,iso-Butane,0.15
4,n--Butane,0.12
5,iso-Pentane,0.04
6,n-Pentane,0.12
7,neo-Pentane,0.06
8,n-Hexane,0.04
9,Carbondioxide,0.00


### Input Data Required: 

In [7]:
pressure = float(input("Enter pressure in psia:")) # psia
pressure = pressure/14.7; # atmosphere
temperature = float(input("Enter Temperature in Rankine: ")) # Rankine
temperature = ((temperature-460-32)*0.555)+273.15; # Kelvin
MW = float(input("Enter molecular weight of plus fraction:")) # molecular weight of plus fraction
sp_gravity = float(input("Enter specific gravity pf plus fraction:")) # specific gravity of plus fraction
plus_fraction = float(input("Enter mole fraction of plus fraction: ")) # mole fraction of plus fraction

Enter pressure in psia:350
Enter Temperature in Rankine: 560
Enter molecular weight of plus fraction:100
Enter specific gravity pf plus fraction:0.87
Enter mole fraction of plus fraction: 0


### Plus Fraction critical properties calculation: 

### θ=a(M)^b γ^c  exp[d(M)+eγ+f(M)γ] 

In [8]:
def plus_property(property_name_index):
    return round((plus_parameter.a[property_name_index]*(MW**plus_parameter.b[property_name_index])*(sp_gravity**plus_parameter.c[property_name_index])*exp(plus_parameter.d[property_name_index]*MW+plus_parameter.e[property_name_index]*sp_gravity+plus_parameter.f[property_name_index]*MW*sp_gravity)),2)
    
plus_Pci = plus_property(1)
plus_Tci = plus_property(0) - 460 # in Fehrenheit
plus_Tbi = plus_property(3) - 460 # in Fehrenheit

### Adding all these calculated parameter to the respective dataframe:

In [9]:
component_added = component_properties.append({'component':'Plus Fraction','molecular_weight':MW,'Tbi':plus_Tbi,"Pci":plus_Pci,"Tci":plus_Tci}, ignore_index = True)
composition_added = composition.append({'Component':'Plus_fraction','Zi':plus_fraction},ignore_index=True)

### Data conversion: 

In [10]:
Tbi = (component_added['Tbi']-32)*0.555 + 273.15 # Fehrenheit to Kelvin
Tci = (component_added['Tci']-32)*0.555 + 273.15 # Fehrenheit to Kelvin
Pci = component_added['Pci']/14.7 # psi to atmospheric pressure

### Constant parameter calculation: 

In [11]:
omegai = (3*Pci.apply(math.log10)/(7*((Tci/Tbi)-1)))-1

omega_a = 0.45724;
omega_b = 0.0778;
R = 8.314; # Universal Gas constant

ai = omega_a*(R**2)*(Tci**2)/(Pci*101000) # here Pci multiplied by 10100 just to convert pressure from atmospheric to Pascal
bi = omega_b*R*Tci/(Pci*101000)
Tri = temperature/Tci

for i in omegai:
    if i<0.49:
        mi = 0.3796+1.54226*omegai-0.2699*omegai**2
    else:
        mi = 0.379642+1.48503*omegai-0.164423*omegai**2+0.016667*omegai**3
        
alphai = (1 + mi*(1-Tri**0.5))**2
ai_alphai = ai*alphai

### Equilibrium Ration by Wilson Correlation:

In [12]:
Ki_wilson = (Pci/pressure)*(5.37*(1+omegai)*(1-Tci/temperature)).apply(math.exp)

### Calculation of nv, nl, Xi, Yi for the first iteration:

In [13]:
# nv = symbols('nv')

# f_nv = lambda nv:sum(composition_added.Zi*(Ki_wilson - 1)/(nv*(Ki_wilson - 1)+1))
# df_nv = lambda nv:-sum(composition_added.Zi*((Ki_wilson-1)**2)/((nv*(Ki_wilson-1) + 1)**2))

# A = sum(composition_added.Zi*(Ki_wilson-1))
# B = sum(composition_added.Zi*(Ki_wilson-1)/Ki_wilson)

# nv = A/(A - B) # initial guess of nv can be best estimated by this equation

# while abs(f_nv(nv))>0.0001:
#     nv = nv - f_nv(nv)/df_nv(nv)

# print(f"Composition in vapor phase is: {nv}")
# nl = 1 - nv
# print(f"Composition in vapor phase is: {1-nv}")
# Xi = composition_added.Zi/(nl + nv*Ki_wilson)
# Yi = Xi*Ki_wilson

### Calculation of parameters needed to calaculate compressibility factor: 

In [14]:
# def a_alpha(composition):
#     return [sum(composition[i]*composition*(ai_alphai[i]*ai_alphai)**0.5) for i in range(len(composition))]

# a_alphal = sum(a_alpha(Xi))
# a_alphag = sum(a_alpha(Yi))
# bml = sum(Xi*bi)
# bmg = sum(Yi*bi)

### Compressibility Factor Calculation: 

In [15]:
# Al = a_alphal*pressure*101000/(R*temperature)**2
# Bl = bml*pressure*101000/(R*temperature)
# Ag = a_alphag*pressure*101000/(R*temperature)**2
# Bg = bmg*pressure*101000/(R*temperature)

# Zl, Zg = symbols("Zl Zg")
# f_Zl = Zl**3+(Bl-1)*(Zl)**2+(Al-3*(Bl)**2-2*Bl)*Zl-(Al*Bl-(Bl)**2-(Bl)**3)
# f_Zg = Zg**3+(Bg-1)*(Zg)**2+(Ag-3*(Bg)**2-2*Bg)*Zg-(Ag*Bg-(Bg)**2-(Bg)**3)
# Zl = solve(f_Zl,Zl)
# Zg = solve(f_Zg,Zg)

# # eliminate complex roots:
# Zl = min([i for i in Zl if sp.im(i)==0]) # compressibitly factor of liquid phase
# Zg = max([i for i in Zg if sp.im(i)==0]) # compressibility factor of gas phase
# print(f"Zg: {round(Zg,3)}")
# print(f"Zl: {round(Zl,3)}")

### Fugacity Calculation: 

In [16]:
# def psi(composition):
#     psi = [sum(composition*((ai_alphai[i]*ai_alphai)**0.5)) for i in range(len(composition))]
#     return pd.Series(psi)  
# psil = psi(Xi)
# psig = psi(Yi)

# # Fugacity of Liquid phase:
# phil = (bi*(Zl-1)/bml)-math.log(Zl-Bl)-(Al/(2*(2**0.5)*Bl))*((2*psil/a_alphal)-(bi/bml))*math.log((Zl+(1+(2**0.5))*Bl)/(Zl+(1-(2**0.5))*Bl))
# phil = pd.Series([math.exp(i) for i in phil])

# # Fugacity of Vapor phase:
# phig = (bi*(Zg-1)/bmg)-math.log(Zg-Bg)-(Ag/(2*(2**0.5)*Bg))*((2*psig/a_alphag)-(bi/bmg))*math.log((Zg+(1+(2**0.5))*Bg)/(Zg+(1-(2**0.5))*Bg))
# phig = pd.Series([math.exp(i) for i in phig])

# # Equilibrium ratio from PREOS:

# Kip = phil/phig
# result = sum(((Kip/Ki_wilson)-1)**2) # Condition to converge the value to get iteration
# result

### Iterations: 

In [17]:
result = 0.1
Kip = Ki_wilson
while result>0.000000000000001:
    
    Ki_wilson = Kip
    nv = symbols('nv')

    f_nv = lambda nv:sum(composition_added.Zi*(Ki_wilson - 1)/(nv*(Ki_wilson - 1)+1))
    df_nv = lambda nv:-sum(composition_added.Zi*((Ki_wilson-1)**2)/((nv*(Ki_wilson-1) + 1)**2))

    A = sum(composition_added.Zi*(Ki_wilson-1))
    B = sum(composition_added.Zi*(Ki_wilson-1)/Ki_wilson)

    nv = A/(A - B) # initial guess of nv can be best estimated by this equation

    while abs(f_nv(nv))>0.0001:
        nv = nv - f_nv(nv)/df_nv(nv)

    nl = 1 - nv
    Xi = composition_added.Zi/(nl + nv*Ki_wilson)
    Yi = Xi*Ki_wilson

    def a_alpha(composition):
        return [sum(composition[i]*composition*(ai_alphai[i]*ai_alphai)**0.5) for i in range(len(composition))]

    a_alphal = sum(a_alpha(Xi))
    a_alphag = sum(a_alpha(Yi))
    bml = sum(Xi*bi)
    bmg = sum(Yi*bi)

    Al = a_alphal*pressure*101000/(R*temperature)**2
    Bl = bml*pressure*101000/(R*temperature)
    Ag = a_alphag*pressure*101000/(R*temperature)**2
    Bg = bmg*pressure*101000/(R*temperature)

    Zl, Zg = symbols("Zl Zg")
    f_Zl = Zl**3+(Bl-1)*(Zl)**2+(Al-3*(Bl)**2-2*Bl)*Zl-(Al*Bl-(Bl)**2-(Bl)**3)
    f_Zg = Zg**3+(Bg-1)*(Zg)**2+(Ag-3*(Bg)**2-2*Bg)*Zg-(Ag*Bg-(Bg)**2-(Bg)**3)
    Zl = solve(f_Zl,Zl)
    Zg = solve(f_Zg,Zg)

    # eliminate complex roots:
    Zl = min([i for i in Zl if sp.im(i)==0]) # compressibitly factor of liquid phase
    Zg = max([i for i in Zg if sp.im(i)==0]) # compressibility factor of gas phase

    def psi(composition):
        psi = [sum(composition*((ai_alphai[i]*ai_alphai)**0.5)) for i in range(len(composition))]
        return pd.Series(psi)  
    psil = psi(Xi)
    psig = psi(Yi)

    # Fugacity of Liquid phase:
    phil = (bi*(Zl-1)/bml)-math.log(Zl-Bl)-(Al/(2*(2**0.5)*Bl))*((2*psil/a_alphal)-(bi/bml))*math.log((Zl+(1+(2**0.5))*Bl)/(Zl+(1-(2**0.5))*Bl))
    phil = pd.Series([math.exp(i) for i in phil])

    # Fugacity of Vapor phase:
    phig = (bi*(Zg-1)/bmg)-math.log(Zg-Bg)-(Ag/(2*(2**0.5)*Bg))*((2*psig/a_alphag)-(bi/bmg))*math.log((Zg+(1+(2**0.5))*Bg)/(Zg+(1-(2**0.5))*Bg))
    phig = pd.Series([math.exp(i) for i in phig])

    # Equilibrium ratio from PREOS:

    Kip = phil/phig
    result = sum(((Kip/Ki_wilson)-1)**2) # Condition to converge the value to get iteration
result

8.776743617740491e-18

### Calculated Result: 

In [18]:
print(f"Composition in vapor phase(nv): {round(nv,3)}")
print(f"Composition in liquid phase(nl): {round(nl,3)}")
print(f"Compressibility fator in gas phase(Zg): {round(Zg,3)}")
print(f"Compressibility fator in liquid phase(Zl): {round(Zl,3)}")

Composition in vapor phase(nv): 0.162
Composition in liquid phase(nl): 0.838
Compressibility fator in gas phase(Zg): 0.851
Compressibility fator in liquid phase(Zl): 0.091


In [21]:
result = pd.DataFrame({"Component":component_added.component,"Molecular Weight":component_added.molecular_weight, "Composition":composition_added.Zi,"Liquid Composition":round(Xi,4), "Gas Composition":round(Yi,4), "Equilibrium Ratio":round(Kip,4)})
result

,Component,Molecular Weight,Composition,Liquid Composition,Gas Composition,Equilibrium Ratio
0,Methane,16.0430,0.15,0.0784,0.5212,6.6496
1,Ethane,30.0700,0.17,0.1521,0.2628,1.7282
2,Propane,44.1000,0.15,0.1595,0.1007,0.6311
3,iso-Butane,58.1200,0.15,0.1690,0.0517,0.3058
4,n-Butane,58.1200,0.12,0.1370,0.0318,0.2322
5,iso-Pentane,72.1500,0.04,0.0467,0.0052,0.1109
6,n-Pentane,72.1500,0.12,0.1407,0.0125,0.0890
7,neo-Pentane,72.1500,0.06,0.0692,0.0124,0.1799
8,n-Hexane,86.1800,0.04,0.0474,0.0017,0.0352
9,Carbondioxide,44.0100,0.00,0.0000,0.0000,2.2917


### Gas Density:

In [22]:
Mag = sum(Yi*component_added.molecular_weight) # apparent molecular weight of gas phase
rhog = round(((pressure*101000*Mag)/(1000*Zg*R*temperature)),3) # Density of Gas in kg/m3
print(f"Density of Gas: {rhog*2.2/3.28**3} pcf")

Density of Gas: 1.89958734200023 pcf


### Liquid Density: 

In [23]:
Mal = sum(Xi*component_added.molecular_weight) # apparent molecular weight of gas phase
rhol = round(((pressure*101000*Mal)/(1000*Zl*R*temperature)),3) # Density of Gas in pcf
print(f"Density of Liquid:{rhol*2.2*5.615/((3.28**3)*42)} ppg")

Density of Liquid:4.52434646585563 ppg
